## IMPORT LIBRARIES

In [1]:
# data wrangling

import numpy as np
import pandas as pd
import re

# data visualization

import matplotlib.pyplot as plt
import seaborn as sns

# text processing

import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition

# filter warnings

import warnings
warnings.filterwarnings('ignore')

## OVERVIEW

In [2]:
# set pandas option

pd.set_option('display.max_colwidth', -1)

In [3]:
# load data

df = pd.read_csv('consumer_compliants.csv')

In [4]:
# show top 5

df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Company has responded to the consumer and the CFPB and chooses not to provide a public response,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter 

In [5]:
# show info()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57453 entries, 0 to 57452
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date received                 57453 non-null  object 
 1   Product                       57453 non-null  object 
 2   Sub-product                   57453 non-null  object 
 3   Issue                         57453 non-null  object 
 4   Sub-issue                     57453 non-null  object 
 5   Consumer complaint narrative  57453 non-null  object 
 6   Company public response       57453 non-null  object 
 7   Company                       57453 non-null  object 
 8   State                         57453 non-null  object 
 9   ZIP code                      57453 non-null  object 
 10  Tags                          57453 non-null  object 
 11  Consumer consent provided?    57453 non-null  object 
 12  Submitted via                 57453 non-null  object 
 13  D

In [6]:
# show null values

df.isna().sum()

Date received                   0    
Product                         0    
Sub-product                     0    
Issue                           0    
Sub-issue                       0    
Consumer complaint narrative    0    
Company public response         0    
Company                         0    
State                           0    
ZIP code                        0    
Tags                            0    
Consumer consent provided?      0    
Submitted via                   0    
Date sent to company            0    
Company response to consumer    0    
Timely response?                0    
Consumer disputed?              57453
Complaint ID                    0    
dtype: int64

In [7]:
# setting new dataframe

df_new = df[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'Complaints'})

In [8]:
# show top 5 new data

df_new.head()

,Complaints,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never

In [9]:
# train test split

X_train, X_test = train_test_split(df_new, test_size=0.3, random_state=42)

In [10]:
# show train example

X_train['Product'].value_counts()

Debt collection                15254
Credit card or prepaid card    9237 
Mortgage                       6826 
Checking or savings account    4910 
Student loan                   2071 
Vehicle loan or lease          1919 
Name: Product, dtype: int64

In [11]:
# show test example

X_test['Product'].value_counts()

Debt collection                6518
Credit card or prepaid card    3956
Mortgage                       2973
Checking or savings account    2093
Student loan                   879 
Vehicle loan or lease          817 
Name: Product, dtype: int64

In [12]:
# set stemmer using PorterSemmer

stemmer = PorterStemmer()

In [13]:
# function to automate text tokenization

def tokenize(text):
    tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2)]
    stems  = [stemmer.stem(item) for item in tokens]
    return stems

In [14]:
# set vectorizer using tf-idf

vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', max_df=0.75, max_features=10000, use_idf=False, norm=None)
tf_vectors = vectorizer_tf.fit_transform(X_train['Complaints']) 

In [15]:
# show vecorizer features

vectorizer_tf.get_feature_names()

["'account",
 "'all",
 "'annual",
 "'back",
 "'busi",
 "'collect",
 "'credit",
 "'debt",
 "'disput",
 "'find",
 "'fraud",
 "'investig",
 "'late",
 "'make",
 "'may",
 "'new",
 "'not",
 "'pend",
 "'process",
 "'secur",
 "'sign",
 "'system",
 "'the",
 "'they",
 "'thi",
 "'you",
 '-.02',
 '-3-501',
 '-and',
 '-call',
 '-origin',
 '-pleas',
 '-proof',
 '-see',
 '-she',
 '-that',
 '-the',
 '-they',
 '-thi',
 '-what',
 '-which',
 '-your',
 '....',
 '.after',
 '.all',
 '.also',
 '.and',
 '.becaus',
 '.but',
 '.call',
 '.even',
 '.for',
 '.gov',
 '.how',
 '.must',
 '.not',
 '.now',
 '.pdf',
 '.pleas',
 '.she',
 '.thank',
 '.that',
 '.the',
 '.then',
 '.there',
 '.they',
 '.thi',
 '.well',
 '.what',
 '.when',
 '.which',
 '/lender',
 '0.00',
 '0.01',
 '0.125',
 '0.25',
 '1,000',
 '1,000,000',
 '1-1/2',
 '1-800',
 '1-month',
 '1-year',
 '1.00',
 '1.25',
 '1.30',
 '1.59',
 '1.75',
 '1.99',
 '1/12',
 '10,000',
 '10-1-36',
 '10-1-36.',
 '10-12',
 '10-14',
 '10-15',
 '10-20',
 '10-busi',
 '10-day',
 '

In [16]:
# modelling using LDA

lda = decomposition.LatentDirichletAllocation(n_components=6, max_iter=5, learning_method='online', learning_offset=50, n_jobs=-1, random_state=42)

# fit model prediction

W1 = lda.fit_transform(tf_vectors)
H1 = lda.components_

In [17]:
# show prediction

W1

array([[5.15548093e-02, 2.14103085e-01, 1.78452342e-03, 5.59450412e-01,
        1.77795960e-03, 1.71329210e-01],
       [3.32130789e-01, 7.19386746e-02, 3.37323471e-01, 2.57570612e-01,
        5.18242839e-04, 5.18210257e-04],
       [2.30933479e-03, 1.02331101e-01, 1.78345205e-01, 6.65983017e-01,
        4.87314191e-02, 2.29992364e-03],
       ...,
       [3.90983093e-03, 7.41005925e-02, 4.37834942e-01, 3.92948260e-03,
        4.76288670e-01, 3.93648185e-03],
       [3.91989310e-03, 3.91886601e-03, 1.34431036e-01, 8.49921117e-01,
        3.91348872e-03, 3.89559945e-03],
       [1.01439328e-01, 8.85910659e-01, 3.16820966e-03, 3.16546662e-03,
        3.16154926e-03, 3.15478726e-03]])

In [18]:
# show topic with top 10 words

num_words=10

vocab = np.array(vectorizer_tf.get_feature_names())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

In [19]:
# show topics

topics

['thi receiv told email time servic said ask phone custom',
 'loan mortgag thi home payment year compani time servic insur',
 'account card bank thi charg credit check money close chase',
 'thi credit debt report collect account compani receiv letter inform',
 'payment late credit account balanc month paid thi statement charg',
 'thi debt report provid inform request document account consum violat']

In [20]:
# getting prediction to data train

colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train['Complaints']))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [21]:
# show prediction on data train

df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.05,0.21,0.00,0.56,0.00,0.17,3
Doc1,0.33,0.07,0.34,0.26,0.00,0.00,2
Doc2,0.00,0.10,0.18,0.67,0.05,0.00,3
Doc3,0.00,0.02,0.88,0.07,0.04,0.00,2
Doc4,0.03,0.03,0.03,0.83,0.03,0.03,3
...,...,...,...,...,...,...,...
Doc40212,0.68,0.29,0.00,0.03,0.00,0.00,0
Doc40213,0.00,0.00,0.50,0.00,0.00,0.49,2
Doc40214,0.00,0.07,0.44,0.00,0.48,0.00,4
Doc40215,0.00,0.00,0.13,0.85,0.00,0.00,3


In [22]:
X_train.head()

Complaints  \
42027  After many weeks of receiving hourly phone calls from Navient, I sent them a letter instructing them only to contact me by mail, per the Fair Debt Collection Practices Act, and the calls stopped, although they have emailed me a couple of times. Today, XX/XX/19, the mother of a friend from middle school let me know that Navient had called HER and left a message for me. She is not listed as an emergency or secondary contact in any of my paperwork ; Navient should not have her number unless they dig it up themselves. This is a huge breach of privacy and a clear violation of the FDCPA, given that I had instructed them to only contact me by phone and that they contacted someone completely unrelated to me and told them they were looking for me regarding student loan debt. Per the FDCPA : " a debt collector may not communicate, in connection with the collection of any debt, with any person other than the consumer, his attorney, a consumer reporting agency if otherwise permitted by law, the creditor, the attorney of the creditor, or the attorney of the debt collector. '' Navient clearly violated this. \n\nThe person who contacted her was named XXXX XXXX, and she gave the reference number XXXX and the phone number XXXX extension XXXX.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [23]:
# predict model to test

WHold = lda.transform(vectorizer_tf.transform(X_test['Complaints']))

In [25]:
# getting model to data test

colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_test['Complaints']))]
df_doc_topic = pd.DataFrame(np.round(WHold, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [27]:
# show prediction on data test

df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.18,0.00,0.08,0.74,0.01,0.00,3
Doc1,0.12,0.64,0.09,0.04,0.07,0.04,1
Doc2,0.22,0.70,0.07,0.00,0.00,0.00,1
Doc3,0.02,0.02,0.02,0.71,0.02,0.21,3
Doc4,0.01,0.01,0.01,0.01,0.01,0.94,5
...,...,...,...,...,...,...,...
Doc17231,0.07,0.00,0.53,0.00,0.38,0.02,2
Doc17232,0.00,0.00,0.00,0.58,0.00,0.41,3
Doc17233,0.00,0.00,0.60,0.38,0.00,0.00,2
Doc17234,0.51,0.06,0.43,0.00,0.00,0.00,0


In [28]:
X_test.head()

,Complaints,Product,Company
2710,"On XX/XX/XXXX, I paid off the account in full to the original creditor. During this phone call, the original creditor informed me that they will close the account with the debt collection agency. \n\nOn XX/XX/XXXX, I called the original creditor regarding the account closure because I will still seeing the debt on my credit report. During this phone call, they said they had emailed the supervisor of the collection agency on XX/XX/XXXX to close the account and that the collection account will be closed soon. \n\nOn XX/XX/XXXX, I called the supervisor of the collection agency who informed me that he never received any communication from the original creditor regarding the closure of this account. He informed me that he needed a formal confirmation from the original creditor in order to close the account. I placed the supervisor of the collection agency on a 3-way phone call with the original creditor and myself. During this phone call, the supervisor told the original creditor that no communication has been received from the original creditor regarding this account while the original creditor claims she has submitted it. The supervisor of the collection agency explained the process by which the original creditor must take to close the account and both agreed to take the necessary measures to move forward in the closure of the account. \n\nOn XX/XX/XXXX, I called the supervisor of the collection agency who informed me that he never received any communication from the original creditor regarding the closure of this account even after the phone call on XX/XX/XXXX. \n\nOn XX/XX/XXXX, I called the original creditor and asked to be directed to the woman who handles account closure but got her voicemail twice. On the same day, I spoke to another representative of the original creditor who told me the account has been closed as the debt has been paid off. She further suggested that I just needed to call the collection agency and tell them that the debt has been paid off to close my account. I reminded her that I had done so and kept getting nowhere as the communication for closure needed to occur directly between the original creditor and the collection agency. \n\nPlease help me resolve this situation. I paid off my medical debt and just want this off my credit report. I'm not sure how to resolve this situation where the debt has been paid off, but the collection agency refuses to remove the account until the original creditor close it, while the original creditor claims they already closed it.",Debt collection,"CCS Financial Services, Inc."
35615,"I'm in the process of refinancing my exsisting home loan. I've been involved in the process for over a month now. I paid {$500.00} ( on XX/XX/XXXX ) in full for my appraisal. Then my mortgage company came back to me to collect {$300.00} more. So I have TWO issues with Alliant Credit Union -- # 1 that they are charging me more money for an appraisal than what a basic appraisal costs. ( I know this as Ive spoken to the Appraisal company -- XXXX XXXX, XXXX ) # 2. Alliant Credit Union is requiring an "" inside '' appraisal. This should not have to be done for a refinance -- especially for an exsisting loan. AND especially due to the circumstances surrounding the COVID-19 crisis. There are alternative ways to complete the refinance process that does not involve an "" inside '' inspection that would jeopardize the health and safety of my family. I've talked with XXXX XXXX. Who has guidelines published on their website and Selling guides of appraisal exceptions that need to be made. I've been told that a "" drive-by '' ( exterior only ) and/or a desktop appraisal can be done. I've also been told that there is a process titled DRDC -- Directed Remote Data Collection -- which is a process by which an inspection can be completed remotely by an appraiser who guides the homeowner through the inspection aided by video conferencing technology. Again, these are all suitabl